In [ ]:
# L'exécution de cette cellule démarre l'application Spark

In [ ]:
import pandas as pd
import numpy as np
import io
import tensorflow as tf
from PIL import Image
from tensorflow.keras.applications.mobilenet_v2 import MobileNetV2, preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras import Model
from pyspark.sql.functions import col, pandas_udf, PandasUDFType, element_at, split

In [ ]:
PATH = 's3://oc-p9-data'
PATH_Data = PATH+'/EMR/Test'
PATH_Result = PATH+'/EMR/Results'
PATH_Result_PCA = PATH+'/EMR/Results_pca'
print('PATH:        '+\
      PATH+'\nPATH_Data:   '+\
      PATH_Data+'\nPATH_Result: '+PATH_Result)

In [ ]:
images = spark.read.format("binaryFile") \
  .option("pathGlobFilter", "*.jpg") \
  .option("recursiveFileLookup", "true") \
  .load(PATH_Data)

In [ ]:
images = images.withColumn('label', element_at(split(images['path'], '/'),-2))
print(images.printSchema())
print(images.select('path','label').show(5,False))

In [ ]:
model = MobileNetV2(weights='imagenet',
                    include_top=True,
                    input_shape=(224, 224, 3))

In [ ]:
new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)

In [ ]:
brodcast_weights = sc.broadcast(new_model.get_weights())

In [ ]:
new_model.summary()

In [ ]:
def model_fn():
    """
    Returns a MobileNetV2 model with top layer removed 
    and broadcasted pretrained weights.
    """
    model = MobileNetV2(weights='imagenet',
                        include_top=True,
                        input_shape=(224, 224, 3))
    for layer in model.layers:
        layer.trainable = False
    new_model = Model(inputs=model.input,
                  outputs=model.layers[-2].output)
    new_model.set_weights(brodcast_weights.value)
    return new_model

In [ ]:
def preprocess(content):
    """
    Preprocesses raw image bytes for prediction.
    """
    img = Image.open(io.BytesIO(content)).resize([224, 224])
    arr = img_to_array(img)
    return preprocess_input(arr)

def featurize_series(model, content_series):
    """
    Featurize a pd.Series of raw images using the input model.
    :return: a pd.Series of image features
    """
    input = np.stack(content_series.map(preprocess))
    preds = model.predict(input)
    # For some layers, output features will be multi-dimensional tensors.
    # We flatten the feature tensors to vectors for easier storage in Spark DataFrames.
    output = [p.flatten() for p in preds]
    return pd.Series(output)

@pandas_udf('array<float>', PandasUDFType.SCALAR_ITER)
def featurize_udf(content_series_iter):
    '''
    This method is a Scalar Iterator pandas UDF wrapping our featurization function.
    The decorator specifies that this returns a Spark DataFrame column of type ArrayType(FloatType).

    :param content_series_iter: This argument is an iterator over batches of data, where each batch
                              is a pandas Series of image data.
    '''
    # With Scalar Iterator pandas UDFs, we can load the model once and then re-use it
    # for multiple data batches.  This amortizes the overhead of loading big models.
    model = model_fn()
    for content_series in content_series_iter:
        yield featurize_series(model, content_series)

In [ ]:
# spark.conf.set("spark.sql.execution.arrow.maxRecordsPerBatch", "1024")

In [ ]:
features_df = images.repartition(24).select(col("path"),
                                            col("label"),
                                            featurize_udf("content").alias("features")
                                           )

In [ ]:
print(PATH_Result)

In [ ]:
features_df.cache()
features_df.count()

In [ ]:
features_df.write.mode("overwrite").parquet(PATH_Result)

**Let's reduce the output vector using PCA**
- We currently have a spark dataframe that have a feature column that contain arrays
- We need to change from array to dense vectors that is the format needed for PCA
- let's create a simple UDF that take care of that !!

In [ ]:
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors, VectorUDT
from pyspark.sql.functions import udf

In [ ]:
def array_to_vector(array):
    return Vectors.dense(array)

array_to_vector_udf = udf(array_to_vector, VectorUDT())

features_df = features_df.withColumn("features", array_to_vector_udf(features_df["features"]))

In [ ]:
num_components = 50

pca = PCA(k=num_components, inputCol="features", outputCol="pca_features")
pca_model = pca.fit(features_df)
df_pca = pca_model.transform(features_df)

In [ ]:
df_pca.show()

In [ ]:
df_pca.write.mode("overwrite").parquet(PATH_Result_PCA)

In [ ]:
first_row_pca = df_pca.limit(1).toPandas()

display(first_row_pca)

In [ ]:
features_length = len(first_row_pca['features'][0])
pca_features_length = len(first_row_pca['pca_features'][0])

print(f"Length of 'features' vector: {features_length}")
print(f"Length of 'pca_features' vector: {pca_features_length}")

In [ ]:
explained_variances = pca_model.explainedVariance
cumulative_variance = explained_variances.toArray().cumsum()
print(f"Cumulative variance explained by the first 50 components: {cumulative_variance[49]:.4f}")